# Bài tập về nhà môn thị giác máy
Nhận diện giới tính và dộ tuổi của em bé <10 tuổi    
Họ và Tên sv:   
Nguyễn Bá Huy - MSSV:19146194                       
  
Notebook: này dùng để dự đoán giới tính                          
  
Mô tả :                                                                      
Trong notebook này,Em training độ tuổi, ảnh training là ảnh xám kích thước 200x200                    
Dataset : https://www.kaggle.com/datasets/nguyenbahuy/gender-dataset/settings 

In [ ]:

from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop,SGD
from keras.callbacks import EarlyStopping 
import matplotlib as plt
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Gọi các thư viện cần thiết
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split # Chia dữ liệu ra làm 2 phần
from keras.layers import Dense, Activation, Dropout, BatchNormalization,LSTM # LSTM biên dạng ANN, BatchNormalization: cho nhỏ lại
from keras.models import Sequential
from tensorflow.keras.optimizers import 

from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score # Để đo lường

from keras.utils import np_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

import tensorflow as tf
import numpy as np
import cv2
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import keras 
from keras.layers import *
from keras.models import *
from keras import backend as K
from tensorflow.keras.utils import load_img, img_to_array


In [ ]:
target_size=(200,200) #Kích thước ảnh đầu ra mong muốn
batch_size=128
epochs=10
verbose=1 #Số batch_size
directory="../input/d/nguyenbahuy/gender-dataset" #Đường dẫn tới thư mục chứa hình ảnh 
num_classes=2 #Số lượng class đầu ra 
img_size=200 #Kích thước ảnh đưa vào train mô hình
learning_rate =0.001
momentum = 0.9
optimizer='adam'
loss='mse'
color_mode = 'grayscale'
class_mode='categorical'

# Hàm con
Chia data thành tập train và tập validation           

In [ ]:
def generate_data():
    # Preprocessing
    image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)   

    train_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                        directory=directory,color_mode = color_mode,
                                                        shuffle=True,
                                                        target_size=target_size, 
                                                        subset="training",
                                                        class_mode=class_mode)

    validation_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                            directory=directory,color_mode = color_mode,
                                                            shuffle=True,
                                                            target_size=target_size, 
                                                            subset="validation",
                                                            class_mode=class_mode)
    return train_dataset,validation_dataset

# BUILD MÔ HÌNH 

In [ ]:
def get_baseline_model(img_size,num_classes):
    inputs = Input(shape=(img_size,img_size,1))
    #Khởi tạo 2 lớp tích chập
    conv1 = Conv2D(32, kernel_size=(3, 3),activation='relu')(inputs)
    conv2 = Conv2D(64, kernel_size=(3, 3),activation='relu')(conv1)
    #Normalize để các giá trị về khoảng 0-1 giúp nhanh hội tụ
    bn=BatchNormalization()(conv2)
    #Maxpooling giảm kích thước ảnh 2 lần
    pool1 = MaxPooling2D(pool_size=(2, 2))(bn)
    conv3 = Conv2D(128, kernel_size=(3, 3),activation='relu')(pool1)
    conv4 = Conv2D(256, kernel_size=(3, 3),activation='relu')(conv3)
    bn=BatchNormalization()(conv4)
    #Dùng hàm relu
    activation=Activation('relu')(bn)
    
    conv6 = Conv2D(512, kernel_size=(3, 3),activation='relu')(activation)
    bn=BatchNormalization()(conv6)
    activation=Activation('relu')(bn)

    pool2 = MaxPooling2D(pool_size=(2, 2))(activation)

    x = Dropout(0.25)(pool2)


    flat = Flatten()(x) #Đưa về dạng vector 1D

    dropout = Dropout(0.5) #Drop out tránh overfiting
    age_model = Dense(128, activation='relu')(flat)
    age_model = dropout(age_model)
    age_model = Dense(64, activation='relu')(age_model)
    age_model = dropout(age_model)
    age_model = Dense(32, activation='relu')(age_model)
    age_model = dropout(age_model)
    age_model = Dense(num_classes, activation='softmax')(age_model)
    #Tạo model
    model = Model(inputs=inputs, outputs=[age_model])
    return model 

# Training model    

In [10]:
#Khởi tạo tập train và tập valid
train_dataset,validation_dataset=generate_data()
#Xem label 
print(train_dataset.class_indices)
#Khởi tạo mô hình
model = get_baseline_model(img_size,num_classes)
# Hàm tối ưu 
#Sử dụng hàm tối ưu SGD
opt = SGD(learning_rate =learning_rate, momentum = momentum )
model.compile(optimizer = opt, loss ='categorical_crossentropy', metrics = ['accuracy'])
#Training mô hình
history=model.fit(train_dataset,batch_size=batch_size,epochs=epochs,verbose=verbose,validation_data=validation_dataset)
# Lưu lại weights của mô hình
model.save('gender_200x200.h5')

Epoch 1/10
1477/1477 [==============================] - 356s 241ms/step - loss: 0.5643 - accuracy: 0.7229 - val_loss: 0.4959 - val_accuracy: 0.8212
Epoch 2/10
1477/1477 [==============================] - 347s 235ms/step - loss: 0.4942 - accuracy: 0.7802 - val_loss: 0.4541 - val_accuracy: 0.8255
Epoch 3/10
1477/1477 [==============================] - 348s 235ms/step - loss: 0.4236 - accuracy: 0.8246 - val_loss: 0.5340 - val_accuracy: 0.7087
Epoch 4/10
1477/1477 [==============================] - 347s 235ms/step - loss: 0.3777 - accuracy: 0.8493 - val_loss: 0.3886 - val_accuracy: 0.8339
Epoch 5/10
1477/1477 [==============================] - 348s 235ms/step - loss: 0.3368 - accuracy: 0.8691 - val_loss: 0.2839 - val_accuracy: 0.9053
Epoch 6/10
1477/1477 [==============================] - 347s 235ms/step - loss: 0.3098 - accuracy: 0.8835 - val_loss: 0.3587 - val_accuracy: 0.8367
Epoch 7/10
1477/1477 [==============================] - 347s 235ms/step - loss: 0.2877 - accuracy: 0.8938 - val_

In [ ]:
# vẽ lại quá trình học
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()

In [ ]:
import numpy as np
#File tới hình ảnh để test
filename = "../input/data-fm-id-gd/Detect_faceMask_gender_indentified/dhung_nomask/dhung_nomask (259).jpg"
img = load_img(filename,target_size=(200,200))
img_show = plt.imshow(img)
plt.show()
img = img_to_array(img)
img = img.reshape(1,200,200,3)
img = img.astype('float32')
img = img/255
kq= np.argmax(model.predict(img),axis=-1)
if(kq==0):
    print("female")
if(kq==1):
    print("female")
